In [26]:
import torch
from collections import OrderedDict
import os
import json


model_paths = {
    "finance": "/data/bhpeng/SLM-llama/outputs/finance2/finance/checkpoint-195/pytorch_model_retriever.bin",
    "history": "/data/bhpeng/SLM-llama/outputs/history2/history/checkpoint-4026/pytorch_model_retriever.bin",
    "medical": "/data/bhpeng/SLM-llama/outputs/medical2/medical/pytorch_model_retriever.bin"
}
all_dict_path = "/data/bhpeng/SLM-llama/outputs/finance_history_medical"

In [31]:
state_dict = dict()
keys = []
weight_offset = []
for k, v in model_paths.items():
    state_dict[k] = torch.load(v)
    if k == 'medical':
        state_dict[k]['retriever.weight_offset'] = state_dict[k]['retriever.weight_offset'][:, :4, ...]
        state_dict[k]['retriever.keys'] = state_dict[k]['retriever.keys'][:, :, :4, :]
    print(k, state_dict[k]['retriever.keys'].shape, state_dict[k]['retriever.weight_offset'].shape)
    keys.append(state_dict[k]['retriever.keys'])
    weight_offset.append(state_dict[k]['retriever.weight_offset'])

finance torch.Size([1, 6, 4, 128]) torch.Size([6, 4, 2, 1048576])
history torch.Size([1, 6, 4, 128]) torch.Size([6, 4, 2, 1048576])
medical torch.Size([1, 6, 4, 128]) torch.Size([6, 4, 2, 1048576])


In [32]:

keys = torch.cat(keys, dim=2)
weight_offset = torch.cat(weight_offset, dim=1)
config_all = {
    "task_pool_index_range": {
        "finance": [0, 4],
        "history": [4, 8],
        "medical": [8, 12]
    },
    "pool_size": 12,
    "prompt_topk": 2,
    "low_rank": 8,
    "groups": 6,
    "similarity_type": "cosine"
}
state_dict_all =OrderedDict()
state_dict_all['retriever.weight_offset'] = weight_offset
state_dict_all['retriever.keys'] = keys
torch.save(state_dict_all, os.path.join(all_dict_path, "pytorch_model.bin"))
with open(os.path.join(all_dict_path, "config.json"), "w") as f:
    json.dump(config_all, f, indent=2)